# Visulalizing and Calculating Parameters on a Skew-T

A powerful tool is being developed that will have the same functionality as GEMPAK, but be done in the Python programming langauage. One example of this is the ability to plot Skew-Ts using the Python module [MetPy](https://unidata.github.io/MetPy/latest/index.html). However, there is greatly functionality than that contained in GEMPAK, as you can use MetPy to calculate a whole host of variables from the data plotted on the Skew-T (e.g., CAPE, CIN, LCL, LFC, EL). This notebook is a brief introduction to how that can be done.

To plot a skew-T, you need to specify the date and time (in UTC) and choose an upper-air station location. The locations of current sounding sites can be found at http://weather.rap.ucar.edu/upper

### Import Packages

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.plots import SkewT
from metpy.units import units, pandas_dataframe_to_unit_arrays
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir

### Get Data

In [ ]:
# Set date that you want
# Data goes back to the 1970's
date = datetime(2007, 5, 4, 18)

# Set station ID, there are different stations back in the day
# Current station IDs found at http://weather.rap.ucar.edu/upper
station = 'DDC'

# Use Siphon module to grab data from remote server
df = WyomingUpperAir.request_data(date, station)

In [ ]:
# Create dictionary of unit arrays
data = pandas_dataframe_to_unit_arrays(df)

# Isolate united arrays from dictionary to individual variables
p = data['pressure']
T = data['temperature']
Td = data['dewpoint']
u = data['u_wind']
v = data['v_wind']

### Plot Skew-T

In [ ]:
# Plot a skew-T image

fig = plt.figure(figsize=(10, 10))

skew = SkewT(fig, rotation=45)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')

# Plot wind barbs, skipping every other one
skew.plot_barbs(p[::2], u[::2], v[::2], y_clip_radius=0.03)

# Set sensible axis limits
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 50)

# Add the relevant special lines
skew.plot_dry_adiabats(t0=np.arange(233,555,10)*units.K)
skew.plot_moist_adiabats(colors='tab:green')
skew.plot_mixing_lines(colors='tab:blue', linestyle='dotted')

# Plot some titles
plt.title(f'Station: K{station}', loc='left')
plt.title('Skew-T/Log-p', loc='center')
plt.title(f'{date} UTC', loc='right')

# Show the plot
#plt.savefig('skewt_image.png', bbox_inches='tight', dpi=150)
plt.show()